In [1]:
import os
import cv2
import xml.etree.ElementTree as ET
import numpy as np

# Define the paths to your image and annotation folders
path_image = "/Users/sumaiyauddin/Documents/Semester03/CCNY-DSE-Capstone-Project-Segmenting-Coral-Branch-tips/data/external/Coral_images/image"
path_annotations = "/Users/sumaiyauddin/Documents/Semester03/CCNY-DSE-Capstone-Project-Segmenting-Coral-Branch-tips/data/external/Coral_images/annotation"

# Initialize lists to store images and masks
images = []
masks = []

# Iterate through each image file in the image folder
for image_filename in os.listdir(path_image):
    if image_filename.lower().endswith(".jpg"):  # Check for image files
        image_path = os.path.join(path_image, image_filename)

        # Load the image using OpenCV
        image = cv2.imread(image_path)

        # Check if the image was loaded successfully
        if image is not None:
            # Resize the image to a consistent size (e.g., 224x224)
            target_size = (224, 224)
            image = cv2.resize(image, target_size)

            # Append the image to the list of images
            images.append(image)

            # Load the corresponding mask (XML file)
            xml_filename = os.path.splitext(image_filename)[0] + ".xml"
            xml_path = os.path.join(path_annotations, xml_filename)

            # Check if the XML file exists
            if os.path.exists(xml_path):
                # Parse the XML file to extract the mask (you may need to adjust this based on your XML structure)
                tree = ET.parse(xml_path)
                root = tree.getroot()

                # Create an empty mask for the current image
                mask = np.zeros(target_size, dtype=np.uint8)

                # Find the "polygon" elements and add their polygons to the mask
                polygons = root.findall(".//object/polygon")

                for polygon in polygons:
                    points = polygon.find("points").text.split()
                    points = [int(p) for p in points]
                    points = np.array(points, dtype=np.int32).reshape((-1, 2))
                    cv2.fillPoly(mask, [points], 1)

                # Append the mask to the list of masks, outside the polygons loop
                masks.append(mask)
            else:
                # Handle the case where the XML mask file is missing
                print(f"XML mask file not found for image: {image_path}")
        else:
            # Handle the case where the image cannot be loaded
            print(f"Image not loaded successfully: {image_path}")


In [11]:
# Convert the image and mask lists to NumPy arrays
images = np.array(images)
masks = np.array(masks)

# Now, check the shapes
print(f"Images shape: {images.shape}")
print(f"Mask shape: {mask.shape}")

Images shape: (899, 224, 224, 3)
Mask shape: (224, 224)


In [5]:
import cv2
import numpy as np

# Resize the masks to match the image size (224x224)
resized_masks = []
for mask in masks:
    resized_mask = cv2.resize(mask, (224, 224), interpolation=cv2.INTER_NEAREST)
    resized_masks.append(resized_mask)

# Convert the list of resized masks to a NumPy array
resized_masks = np.array(resized_masks)

print(f"Mask shape: {resized_mask.shape}")

Mask shape: (224, 224)


In [4]:
import os
import numpy as np
import cv2
import xml.etree.ElementTree as ET
from mrcnn.utils import Dataset

class CoralDataset(Dataset):
    def load_coral_dataset(self, dataset_dir, subset):
        # Add coral classes
        self.add_class("coral", 1, "APAL")  # Replace with your class names
        self.add_class("coral", 2, "Pseododiploria")  # Replace with your class names
        
        # Define the path to your image and annotation directories
        images_dir = os.path.join(dataset_dir, "image")
        annotations_dir = os.path.join(dataset_dir, "annotation")

        # List all image files in the directory
        image_files = os.listdir(images_dir)

        # Add images and annotations to the dataset
        for i, image_file in enumerate(image_files):
            if i < 10 and subset == "test":
                continue
            if i >= 10 and subset == "train":
                continue

            image_path = os.path.join(images_dir, image_file)
            annotation_file = image_file.replace(".jpg", ".xml")
            annotation_path = os.path.join(annotations_dir, annotation_file)
            
            # Get image width and height (adjust if needed)
            image = cv2.imread(image_path)
            height, width, _ = image.shape

            self.add_image(
                "coral",
                image_id=i,
                path=image_path,
                annotation=annotation_path,
                width=width,
                height=height
            )

    def load_mask(self, image_id):
        # Load annotations from XML file
        info = self.image_info[image_id]
        annotation_path = info['annotation']
        tree = ET.parse(annotation_path)
        root = tree.getroot()

        # Initialize lists for masks and class_ids
        masks = []
        class_ids = []

        for obj in root.findall('object'):
            class_name = obj.find('name').text
            class_id = self.class_names.index(class_name)

            # Create binary masks
            mask = np.zeros([info['height'], info['width']], dtype=np.uint8)
            for polygon in obj.findall('polygon'):
                points = []
                for pt in polygon.findall('pt'):
                    x = int(pt.find('x').text)
                    y = int(pt.find('y').text)
                    points.append((x, y))

                cv2.fillPoly(mask, [np.array(points)], 1)

            masks.append(mask)
            class_ids.append(class_id)

        # Convert masks to a 3D array
        masks = np.stack(masks, axis=-1)

        return masks, np.array(class_ids)

    def image_reference(self, image_id):
        return self.image_info[image_id]['path']


: 

In [ ]:
hhh